<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2022/blob/main/notebooks/6-model-mi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wstęp

Na podstwie:

+ Kim, J. K., Park, S., Chen, Y., & Wu, C. (2021). Combining non‐probability and probability survey samples through mass imputation. Journal of the Royal Statistical Society: Series A (Statistics in Society), 184(3), 941-963.
+ Yang, J.K. Kim, and Y. Hwang (2021). "Integration of survey data and big observational data for finite population inference using mass imputation", Survey Methodology, 47, 29--58.


# Pakiety i dane

Ładujemy pakiety

In [1]:
library(tidyverse) 

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Pobieramy dane z zajęć

In [2]:
download.file("https://raw.githubusercontent.com/DepartmentOfStatisticsPUE/bi-2021/main/data/popyt-zajecia-dane.csv",
              "popyt-dane.csv")

In [3]:
dane <- read.csv("popyt-dane.csv", stringsAsFactors = FALSE) %>%
    mutate(woj = as.factor(woj),
           zawod_kod2 = as.factor(zawod_kod2))
tail(dane)

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<fct>,<fct>,<int>,<int>,<lgl>,<lgl>,<int>,<int>
20941,NA,a6a20c0f40c36af79446a53cb1af98dac84ca2cf,NA,2,S,G,8,8,NA,1,TRUE,TRUE,2,1
20942,NA,a6a20c0f40c36af79446a53cb1af98dac84ca2cf,NA,2,S,G,8,9,NA,1,TRUE,TRUE,2,1
20943,NA,5ab1d632da4eda181d7a454f71dd69e4433b6aa5,NA,2,D,H,8,9,NA,1,FALSE,TRUE,1,0
20944,NA,a108d5030e205dc1632b5b8b3eddb725821adfa5,NA,2,S,I,8,7,NA,1,TRUE,TRUE,1,0
20945,NA,a108d5030e205dc1632b5b8b3eddb725821adfa5,NA,2,S,I,8,9,NA,1,TRUE,TRUE,15,0
20946,NA,d5c8a81acfb94539956a0d87c37a233e21ec435f,NA,2,M,H,8,7,NA,1,FALSE,TRUE,1,0


Na tę chwilę założymy, że zbiory są rozłączne!

In [4]:
proba_nielosowa <- dane %>% filter(is.na(id_popyt))
proba_losowa <- dane %>% filter(!is.na(id_popyt))

Regresja logistyczna wyjaśniająca zmienną `jedna_zmiana`

In [5]:
model_a <- glm(formula = jedna_zmiana ~ sek + klasa_pr + sekc_pkd + woj + zawod_kod2, 
               data = proba_nielosowa,
               family = binomial(),
               weights = proba_nielosowa$wolne_miejsca_cbop)

y_hat_a <- fitted(model_a)
y_hat_b <- predict(model_a, proba_losowa, type = "response")
summary(y_hat_b)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.04645 0.27988 0.49770 0.53685 0.81273 0.99871 

In [7]:
naiwny <- weighted.mean(proba_nielosowa$jedna_zmiana, proba_nielosowa$wolne_miejsca_cbop)
mi <- weighted.mean(y_hat_b, proba_losowa$waga*proba_losowa$wolne_miejsca)
print(c(naiwny = naiwny, theta_mi = mi))

   naiwny  theta_mi 
0.5204615 0.6264032 
